# Performance Comparison of Different Simulation Modes

This notebook compares the performance of different simulation modes in our STDP-based MNIST classification system.

## Table of Contents:
1. Setup and Configuration
2. Performance Testing Functions
3. Runtime Device with Cython
4. Multi-threaded Runtime
5. C++ Standalone with OpenMP
6. Results Comparison

## 1. Setup and Configuration

First, let's import necessary libraries and define our test configurations.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from brian2 import *
import time
from functions.data import get_labeled_data, get_data_subset
import pandas as pd

# Test configurations
TEST_CONFIGS = [
    {'name': 'Small Dataset', 'size': 1000, 'epochs': 1},
    {'name': 'Medium Dataset', 'size': 5000, 'epochs': 1},
    {'name': 'Full Epoch', 'size': 60000, 'epochs': 1}
]

# Enable interactive plotting
%matplotlib inline

## 2. Performance Testing Functions

Let's define functions to measure performance of different configurations.

In [ ]:
def run_performance_test(config, device='runtime', num_threads=1):
    """Run a single performance test with given configuration."""
    # Import the main script as a module
    import importlib.util
    spec = importlib.util.spec_from_file_location("mnist_stdp", "../diehl_cook_spiking_mnist_brian2.py")
    mnist_stdp = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mnist_stdp)
    
    # Configure
    mnist_stdp.args.train = True
    mnist_stdp.args.test = False
    mnist_stdp.args.train_size = config['size']
    mnist_stdp.args.epochs = config['epochs']
    mnist_stdp.args.device = device
    mnist_stdp.args.num_threads = num_threads
    
    # Measure time
    start_time = time.time()
    mnist_stdp.main()
    end_time = time.time()
    
    return end_time - start_time

## 3. Runtime Device with Cython

Let's test the default runtime device with Cython optimization.

In [ ]:
runtime_results = []
for config in TEST_CONFIGS:
    time_taken = run_performance_test(config, device='runtime', num_threads=1)
    runtime_results.append({
        'config': config['name'],
        'time': time_taken,
        'mode': 'Runtime+Cython'
    })

## 4. Multi-threaded Runtime

Now let's test with multiple threads.

In [ ]:
threaded_results = []
for config in TEST_CONFIGS:
    time_taken = run_performance_test(config, device='runtime', num_threads=8)
    threaded_results.append({
        'config': config['name'],
        'time': time_taken,
        'mode': 'Multi-threaded'
    })

## 5. C++ Standalone with OpenMP

Finally, let's test the C++ standalone device with OpenMP.

In [ ]:
cpp_results = []
for config in TEST_CONFIGS:
    time_taken = run_performance_test(config, device='cpp_standalone', num_threads=8)
    cpp_results.append({
        'config': config['name'],
        'time': time_taken,
        'mode': 'C++Standalone+OpenMP'
    })

## 6. Results Comparison

Let's compare and visualize the results.

In [ ]:
# Combine all results
all_results = pd.DataFrame(runtime_results + threaded_results + cpp_results)

# Create a bar plot
plt.figure(figsize=(12, 6))
configs = all_results['config'].unique()
x = np.arange(len(configs))
width = 0.25

for i, mode in enumerate(all_results['mode'].unique()):
    data = all_results[all_results['mode'] == mode]
    plt.bar(x + i*width, data['time'], width, label=mode)

plt.xlabel('Dataset Size')
plt.ylabel('Time (seconds)')
plt.title('Performance Comparison of Different Simulation Modes')
plt.xticks(x + width, configs)
plt.legend()
plt.tight_layout()
plt.show()

# Print detailed results
print("\nDetailed Results:")
print(all_results.to_string(index=False))